In [12]:
import pyspark
from pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
sc = SparkContext(appName="Evidently")
spark = SparkSession.builder.getOrCreate()

sc.setLogLevel("ERROR")

24/06/29 17:40:05 WARN Utils: Your hostname, dell resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp0s20f3)
24/06/29 17:40:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/06/29 17:40:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load data

In [4]:
df = spark.read.csv(path='../data/dataset.csv', inferSchema=True, header=True)

In [6]:
df.show(5)

+------+-----+------+-----+----------------+---------------+-------------+---+-----+
|cement| slag|flyash|water|superplasticizer|coarseaggregate|fineaggregate|age|csMPa|
+------+-----+------+-----+----------------+---------------+-------------+---+-----+
| 540.0|  0.0|   0.0|162.0|             2.5|         1040.0|        676.0| 28|79.99|
| 540.0|  0.0|   0.0|162.0|             2.5|         1055.0|        676.0| 28|61.89|
| 332.5|142.5|   0.0|228.0|             0.0|          932.0|        594.0|270|40.27|
| 332.5|142.5|   0.0|228.0|             0.0|          932.0|        594.0|365|41.05|
| 198.6|132.4|   0.0|192.0|             0.0|          978.4|        825.5|360| 44.3|
+------+-----+------+-----+----------------+---------------+-------------+---+-----+
only showing top 5 rows



In [7]:
df.count()

1030

In [8]:
df.printSchema()

root
 |-- cement: double (nullable = true)
 |-- slag: double (nullable = true)
 |-- flyash: double (nullable = true)
 |-- water: double (nullable = true)
 |-- superplasticizer: double (nullable = true)
 |-- coarseaggregate: double (nullable = true)
 |-- fineaggregate: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- csMPa: double (nullable = true)



### Splitting the data

In [11]:
df_nan = df.na.drop()
print('Número de linhas antes de remover valores ausentes:', df.count())
print('Número de linhas após remover valores ausentes:', df_nan.count())

Número de linhas antes de remover valores ausentes: 1030
Número de linhas após remover valores ausentes: 1030


In [9]:
df_train, df_test = df.randomSplit(weights=[0.7,0.3], seed=42)

In [10]:
df_train.count()

762

## Data preparation

In [13]:
def func_modulo_prep_dados(df: DataFrame,
                           variaveis_entrada: list[str],
                           variavel_saida: str,
                           tratar_outliers = True,
                           padronizar_dados = True):

    # Vamos gerar um novo dataframe, renomeando o argumento que representa a variável de saída.
    novo_df = df.withColumnRenamed(variavel_saida, 'label')
    
    # Convertemos a variável alvo para o tipo numérico como float (encoding)
    if str(novo_df.schema['label'].dataType) != 'IntegerType':
        novo_df = novo_df.withColumn("label", novo_df["label"].cast(T.FloatType()))
    
    # Listas de controle para as variáveis
    variaveis_numericas = []
    variaveis_categoricas = []
    
    # Se tiver variáveis de entrada do tipo string, convertemos para o tipo numérico
    for coluna in variaveis_entrada:
        
        # Verifica se a variável é do tipo string
        if str(novo_df.schema[coluna].dataType) == 'StringType':
            
            # Definimos a variável com um sufixo
            novo_nome_coluna = coluna + "_num"
            
            # Adicionamos à lista de variáveis categóricas
            variaveis_categoricas.append(novo_nome_coluna)
            
        else:
            
            # Se não for variável do tipo string, então é numérica e adicionamos na lista correspondente
            variaveis_numericas.append(coluna)
            
            # Colocamos os dados no dataframe de variáveis indexadas
            df_indexed = novo_df
            
    # Se o dataframe tiver dados do tipo string, aplicamos a indexação
    # Verificamos se a lista de variáveis categóricas não está vazia
    if len(variaveis_categoricas) != 0:

        # Loop pelas colunas
        for coluna in novo_df:
            
            # Se a variável é do tipo string, criamos, treinamos e aplicamos o indexador
            if str(novo_df.schema[coluna].dataType) == 'StringType':
                
                # Cria o indexador
                indexer = StringIndexer(inputCol=coluna, outputCol=coluna + "_num") 
                
                # Treina e aplica o indexador
                df_indexed = indexer.fit(novo_df).transform(novo_df)
    else:
        # Se não temos mais variáveis categóricas, então colocamos os dados no dataframe de variáveis indexadas
        df_indexed = novo_df
        
    # Se for necessário tratar outliers, faremos isso agora
    if tratar_outliers == True:
        print("\nAplicando o tratamento de outliers...")
        
        # Dicionário
        d = {}
        
        # Dicionário de quartis das variáveis do dataframe indexado (somente variáveis numéricas)
        for col in variaveis_numericas: 
            d[col] = df_indexed.approxQuantile(col, probabilities=[0.01, 0.99], relativeError=0.25) 
        
        # Agora aplicamos transformação dependendo da distribuição de cada variável
        for col in variaveis_numericas:
            
            # Extraímos a assimetria dos dados e usamos isso para tratar os outliers
            skew = df_indexed.agg(F.skewness(df_indexed[col])).collect() 
            skew = skew[0][0]
            
            # Verificamos a assimetria e então aplicamos:
            
            # Transformação de log + 1 se a assimetria for positiva
            if skew > 1:
                indexed = df_indexed.withColumn(col, 
                    F.log(
                        F.when(df[col] < d[col][0], d[col][0])\
                        .when(df_indexed[col] > d[col][1], d[col][1])\
                        .otherwise(df_indexed[col] ) + 1
                    ).alias(col)
                )
                print("\nA variável " + col + " foi tratada para assimetria positiva (direita) com skew =", skew)
            
            # Transformação exponencial se a assimetria for negativa
            elif skew < -1:
                indexed = df_indexed.withColumn(col,
                    F.exp(
                        F.when(df[col] < d[col][0], d[col][0]).when(df_indexed[col] > d[col][1], d[col][1])\
                        .otherwise(df_indexed[col])
                    ).alias(col)
                )
                print("\nA variável " + col + " foi tratada para assimetria negativa (esquerda) com skew =", skew)
                
            # Assimetria entre -1 e 1 não precisamos aplicar transformação aos dados

    # Vetorização
    
    # Lista final de atributos
    lista_atributos = variaveis_numericas + variaveis_categoricas
    
    # Cria o vetorizador para os atributos
    vetorizador = VectorAssembler(inputCols = lista_atributos, outputCol = 'features')
    
    # Aplica o vetorizador ao conjunto de dados
    dados_vetorizados = vetorizador.transform(df_indexed).select('features', 'label')
    
    # Se a flag padronizar_dados está como True, então padronizamos os dados colocando-os na mesma escala
    if padronizar_dados == True:
        print("\nPadronizando o conjunto de dados para o intervalo de 0 a 1...")
        
        # Cria o scaler
        scaler = MinMaxScaler(inputCol = "features", outputCol = "scaledFeatures")

        # Calcula o sumário de estatísticas e gera o padronizador
        global scalerModel
        scalerModel = scaler.fit(dados_vetorizados)

        # Padroniza as variáveis para o intervalo [min, max]
        dados_padronizados = scalerModel.transform(dados_vetorizados)
        
        # Gera os dados finais
        dados_finais = dados_padronizados.select('label', 'scaledFeatures')
        
        # Renomeia as colunas (requerido pelo Spark)
        dados_finais = dados_finais.withColumnRenamed('scaledFeatures', 'features')
        
        print("\nProcesso Concluído!")

    # Se a flag está como False, então não padronizamos os dados
    else:
        print("\nOs dados não serão padronizados pois a flag padronizar_dados está com o valor False.")
        dados_finais = dados_vetorizados
    
    return dados_finais

In [14]:
variaveis_entrada = df.columns[:-1]
variavel_saida = df.columns[-1]

In [15]:
df_final = func_modulo_prep_dados(df, variaveis_entrada, variavel_saida)


Aplicando o tratamento de outliers...

A variável age foi tratada para assimetria positiva (direita) com skew = 3.2644145354168086

Padronizando o conjunto de dados para o intervalo de 0 a 1...

Processo Concluído!


In [16]:
df_final.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|79.99|[1.0,0.0,0.0,0.32...|
|61.89|[1.0,0.0,0.0,0.32...|
|40.27|[0.52625570776255...|
|41.05|[0.52625570776255...|
| 44.3|[0.22054794520547...|
|47.03|[0.37442922374429...|
| 43.7|[0.63470319634703...|
|36.45|[0.63470319634703...|
|45.85|[0.37442922374429...|
|39.29|(8,[0,3,5,7],[0.8...|
|38.07|[0.22054794520547...|
|28.02|[0.22054794520547...|
|43.01|[0.74315068493150...|
|42.33|[0.20091324200913...|
|47.81|[0.46118721461187...|
|52.91|[0.63470319634703...|
|39.36|[0.08584474885844...|
|56.14|[0.54794520547945...|
|40.56|[0.63470319634703...|
|42.62|(8,[0,3,5,7],[0.8...|
+-----+--------------------+
only showing top 20 rows

